In [1]:
!pip install xgboost
!pip install sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df4 = pd.read_csv("data/test/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df5 = pd.read_csv("data/test/click_log.csv")

In [3]:
df2.sort_values('creative_id').head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [4]:
df4.sort_values('creative_id').head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,3,3,\N,5,598,165
2,4,4,\N,5,108,202
3,5,5,\N,5,109,6
4,6,6,\N,5,400,6


In [5]:
df3.head()

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1


In [6]:
df5.head()

,time,user_id,creative_id,click_times
0,20,3131989,645764,1
1,20,3131989,1027422,1
2,20,3131989,1106443,1
3,20,3131989,629802,1
4,59,3131989,2839769,1


In [7]:
df2 = pd.concat([df2, df4])
df2.sort_values('creative_id', inplace=True)
df2 = df2.drop_duplicates(subset=['creative_id'], keep='first')
df2.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,3,3,\N,5,598,165
2,4,4,\N,5,108,202
3,5,5,\N,5,109,6
4,6,6,\N,5,400,6


In [8]:
df3 = pd.concat([df3, df5])
df = pd.merge(df3, df2, left_on='creative_id', right_on='creative_id')
df.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,15,320815,567330,1,504423,30673,3,32638,319
2,11,355089,567330,1,504423,30673,3,32638,319
3,9,363442,567330,1,504423,30673,3,32638,319
4,14,370513,567330,1,504423,30673,3,32638,319


In [9]:
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.head()

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,30920,9,567330,1,504423,3,32638
1,320815,15,567330,1,504423,3,32638
2,355089,11,567330,1,504423,3,32638
3,363442,9,567330,1,504423,3,32638
4,370513,14,567330,1,504423,3,32638


In [10]:
Y1_train, Y2_train, Y1_test, Y2_test = df1.iloc[:800000, 1:2], df1.iloc[:800000, 2:3], df1.iloc[800000:, 1:2], df1.iloc[800000:, 2:3]
Y1_train

,age
0,4
1,10
2,7
3,5
4,4
...,...
799995,5
799996,2
799997,7
799998,4


In [11]:
X = df3.groupby('user_id')['click_times'].sum()
X

user_id
1          14
2          46
3          30
4          29
5          34
           ..
3999996    75
3999997    20
3999998    20
3999999    86
4000000    67
Name: click_times, Length: 1900000, dtype: int64

In [12]:
df["fre_creative_id"] = (df.groupby("user_id")["creative_id"].transform(lambda x: x.value_counts().index[0]))
X_tmp = df.drop_duplicates(subset=['user_id'],keep='first')
X_tmp = X_tmp[['user_id', 'creative_id']]
X = pd.merge(X, X_tmp, left_on='user_id', right_on='user_id')
X

,user_id,click_times,creative_id
0,1,14,71691
1,2,46,13069
2,3,30,599128
3,4,29,72989
4,5,34,24333
...,...,...,...
1899995,3999996,75,1837000
1899996,3999997,20,3959946
1899997,3999998,20,54150
1899998,3999999,86,649132


In [13]:
# 点击最多的类
X = pd.merge(X, df2[['creative_id', 'product_category']], left_on='creative_id', right_on='creative_id')
X = X.drop('creative_id', axis = 1)
X_tmp = pd.get_dummies(X['product_category'], prefix='category')
X = pd.concat([X.drop('product_category', axis = 1), X_tmp], axis=1)
X = X.sort_values('user_id')
X

,user_id,click_times,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_11,category_12,category_13,category_15,category_16,category_17,category_18
0,1,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
597,2,46,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17622,3,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21685,4,29,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21939,5,34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823716,3999996,75,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1743890,3999997,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1486230,3999998,20,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
652231,3999999,86,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [14]:
X = X.reset_index(drop=True)
X_tmp1 = df3.groupby('user_id')['click_times'].mean()
X_tmp2 = df3.groupby('user_id')['click_times'].std()
# 点击次数的平均值和集中程度
X = pd.merge(X, X_tmp1, left_on='user_id', right_on='user_id')
X = pd.merge(X, X_tmp2, left_on='user_id', right_on='user_id')
X

,user_id,click_times_x,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_11,category_12,category_13,category_15,category_16,category_17,category_18,click_times_y,click_times
0,1,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.076923,0.277350
1,2,46,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.022222,0.149071
2,3,30,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000
3,4,29,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000
4,5,34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.030303,0.174078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899995,3999996,75,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.136364,0.460313
1899996,3999997,20,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000
1899997,3999998,20,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.052632,0.229416
1899998,3999999,86,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.075000,0.265053


In [15]:
# 点击时间分布情况
X_tmp = df[['user_id', 'time']].groupby('user_id')['time'].std()
X = pd.merge(X, X_tmp, left_on='user_id', right_on='user_id')
X

,user_id,click_times_x,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,...,category_11,category_12,category_13,category_15,category_16,category_17,category_18,click_times_y,click_times,time
0,1,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1.076923,0.277350,19.585775
1,2,46,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.022222,0.149071,21.653562
2,3,30,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.000000,0.000000,26.386953
3,4,29,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.000000,0.000000,17.421039
4,5,34,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.030303,0.174078,23.186783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899995,3999996,75,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1.136364,0.460313,24.370547
1899996,3999997,20,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.000000,0.000000,28.349093
1899997,3999998,20,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1.052632,0.229416,27.634519
1899998,3999999,86,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1.075000,0.265053,24.379356


In [16]:
# creave_id 以取模方式进入特征
mo = 20
X_tmp = df.drop_duplicates(subset=['user_id'],keep='first')
X_tmp = X_tmp[['user_id', 'creative_id']]
X_tmp['creative_id'] = X_tmp['creative_id'] % mo
X_tmp

,user_id,creative_id
0,30920,10
1,320815,10
2,355089,10
3,363442,10
4,370513,10
...,...,...
53000276,3845254,14
53122971,833411,19
55393365,404337,3
55643690,482613,14


In [17]:
X_tmp = X_tmp.drop_duplicates(subset=['user_id'], keep='first')
X_tmp = X_tmp.sort_values('user_id')
X_tmp = pd.get_dummies(X_tmp['creative_id'], prefix='creative_id')
X_tmp = X_tmp.reset_index(drop=True)
X = pd.concat([X, X_tmp], axis = 1)

In [18]:
X

,user_id,click_times_x,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,...,creative_id_10,creative_id_11,creative_id_12,creative_id_13,creative_id_14,creative_id_15,creative_id_16,creative_id_17,creative_id_18,creative_id_19
0,1,14,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,46,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,30,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,29,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,34,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899995,3999996,75,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1899996,3999997,20,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1899997,3999998,20,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1899998,3999999,86,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
X_ready = X.drop(['user_id'], axis = 1)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:900000, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.69465125

In [21]:
model1.score(X_test, Y2_test)

0.69295

In [22]:
model2 = xgb.XGBClassifier()
model2.fit(X_train, Y1_train)
model2.score(X_train, Y1_train)

0.24515625

In [23]:
model2.score(X_test, Y1_test)

0.22714

In [24]:
X_output = X_ready[900000:, :]
Y2_output = model1.predict(X_output)
Y1_output = model2.predict(X_output)

In [25]:
Y1_output

array([3, 7, 3, ..., 3, 3, 6])

In [26]:
Y2_output

array([1, 2, 1, ..., 1, 1, 2])

In [27]:
X_output = X[['user_id']][900000:]

In [49]:
import csv
with open('submission.csv', 'w', encoding='utf-8') as csvfile:
    fieldnames = ['user_id', 'predicted_age', 'predicted_gender']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(X_output)):
        writer.writerow({'user_id': str(X_output.iloc[i, 0]), 'predicted_age': str(Y1_output[i]), 
                         'predicted_gender': str(Y2_output[i])})

In [48]:
X_output.shape

(1000000, 1)

In [45]:
X_tmp = df.drop_duplicates(subset=['user_id'],keep='first')
X_tmp = X_tmp[['user_id']]
X_tmp = X_tmp.sort_values('user_id')
X_tmp[900000:]

,user_id
10951437,3000001
244263,3000002
3099563,3000003
23254,3000004
2750936,3000005
...,...
1632471,3999996
6059893,3999997
2678408,3999998
4820478,3999999
